In [2]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt

In [3]:
# load the csv file into the dataframe
users_data = pd.read_csv("BX-Users.csv")

original_count = users_data.shape[0]
print(original_count)

# Display to check some data
users_data.head()


48299


,User-ID,User-City,User-State,User-Country,User-Age
0,8,timmins,ontario,"canada""",NaN
1,9,germantown,tennessee,"usa""",NaN
2,16,albuquerque,new mexico,"usa""",NaN
3,17,chesapeake,virginia,"usa""",NaN
4,19,weston,,NaN,"14"""


In [4]:
# Part 1: UserID
# drop non integer entry
step1_data = users_data[users_data['User-ID'].astype(str).str.match(r'^\d+$')]

# drop the entry for NaN UserID
step1_data = users_data.dropna(subset=['User-ID'])

# delete duplicate data
step1_data = step1_data.drop_duplicates(subset=['User-ID'])

step1_data.shape[0]
step1_data.head()

,User-ID,User-City,User-State,User-Country,User-Age
0,8,timmins,ontario,"canada""",NaN
1,9,germantown,tennessee,"usa""",NaN
2,16,albuquerque,new mexico,"usa""",NaN
3,17,chesapeake,virginia,"usa""",NaN
4,19,weston,,NaN,"14"""


In [5]:
# Part 2: Regional Data (City, State, Country)
# Strip the regional columns off non alphabet characters, then normalize by making it lowercase
step1_data['User-City'] = step1_data['User-City'].replace(r'[^a-zA-Z]', '', regex=True).str.lower()
step1_data['User-State'] = step1_data['User-State'].replace(r'[^a-zA-Z]', '', regex=True).str.lower()
step1_data['User-Country'] = step1_data['User-Country'].replace(r'[^a-zA-Z]', '', regex=True).str.lower()

# replace full white space character or blank character to NaN
step2_data = step1_data.replace(r'^\s*$', np.nan, regex=True)

# change the exact "na" strings to NaN
step2_data = step2_data.replace(r"^na$", np.nan, regex=True)

# show how many NaNs exist
print("NaNs after normalization\n",step2_data[['User-City', 'User-State', 'User-Country']].isna().sum())

# Impute the missing state using other state that have the same city
# Create a mapping from cities to states
city_to_state_map = step2_data.dropna(subset=['User-City', 'User-State']).drop_duplicates(subset=['User-City']).set_index('User-City')['User-State']

# Use the map to fill missing states
step2_data['User-State'] = step2_data['User-State'].fillna(step2_data['User-City'].map(city_to_state_map))

# Impute the missing state using other state that have the same city
# Create a mapping from states to countries
state_to_country_map = step2_data.dropna(subset=['User-State', 'User-Country']).drop_duplicates(subset=['User-State']).set_index('User-State')['User-Country']

# Use the map to fill missing countries
step2_data['User-Country'] = step2_data['User-Country'].fillna(step2_data['User-State'].map(state_to_country_map))

# Check how many NaNs remain
print("\nNaNs after imputation\n",step2_data[['User-City', 'User-State', 'User-Country']].isna().sum())

# drop the remaining NaNs
step2_data = step2_data.dropna(subset=['User-State','User-Country'])

# Make sure no NaNs remain
print("\nNaNs after dropping remaining NaNs\n",step2_data[['User-City', 'User-State', 'User-Country']].isna().sum())

# now that the state and country column is clean, we can drop city columnn since it won't be used
step2_data = step2_data.drop(columns="User-City")
print(step2_data.head())
print("------------------")

# now, use one hot encoding for Country and State
step2_data = pd.get_dummies(step2_data, columns=['User-State', 'User-Country'])

pd.options.display.max_columns = None
step2_data.head()

NaNs after normalization
 User-City         76
User-State      1468
User-Country     672
dtype: int64

NaNs after imputation
 User-City        76
User-State      283
User-Country     57
dtype: int64

NaNs after dropping remaining NaNs
 User-City       39
User-State       0
User-Country     0
dtype: int64
   User-ID User-State User-Country User-Age
0        8    ontario       canada      NaN
1        9  tennessee          usa      NaN
2       16  newmexico          usa      NaN
3       17   virginia          usa      NaN
4       19    florida          usa      14"
------------------


,User-ID,User-Age,User-State_aargau,User-State_aarhus,User-State_abelokipi,User-State_aberdeenshire,User-State_aberta,User-State_abilene,User-State_abruzzo,User-State_acorua,User-State_act,User-State_ae,User-State_ag,User-State_agbs,User-State_ahmedi,User-State_ain,User-State_ak,User-State_akershus,User-State_al,User-State_alabama,User-State_alacant,User-State_alajuela,User-State_alaska,User-State_alava,User-State_albacete,User-State_albay,User-State_alberta,User-State_alentejo,User-State_algarve,User-State_alicante,User-State_almada,User-State_almatyoblast,User-State_alpesmaritimes,User-State_alsace,User-State_altoalentejo,User-State_amsterdam,User-State_andaluca,User-State_andalucia,User-State_andalusia,User-State_andhrapradesh,User-State_angus,User-State_antarctica,User-State_antwerp,User-State_antwerpen,User-State_anywhereusa,User-State_aomoriken,User-State_aomoriprefecture,User-State_ap,User-State_apo,User-State_appenzell,User-State_aquitaine,User-State_ar,User-State_aragn,User-State_aragon,User-State_arizona,User-State_arkansas,User-State_armedforceseurope,User-State_armedforcespacific,User-State_armedserviceseurope,User-State_asia,User-State_asturias,User-State_asturies,User-State_athens,User-State_attika,User-State_attiki,User-State_auckland,User-State_australia,User-State_australiancapitalterritory,User-State_austria,User-State_auvergne,User-State_aveiro,User-State_avon,User-State_bacau,User-State_badajoz,User-State_badenwrrtemberg,User-State_badenwrtemberg,User-State_badenwrttemberg,User-State_badenwuerttemberg,User-State_bahia,User-State_bajacalifornia,User-State_bajacaliforniasur,User-State_baleares,User-State_bali,User-State_balkonien,User-State_bangkok,User-State_bankspeninsula,User-State_banstead,User-State_barcelona,User-State_barcelones,User-State_bari,User-State_barreiro,User-State_basel,User-State_baselland,User-State_baselstadt,User-State_basquecountry,User-State_basquecountryeuskalherria,User-State_basrhin,User-State_bassecote,User-State_bassenormandie,User-State_bavaria,User-State_bavariabayern,User-State_bayern,User-State_bayernbavaria,User-State_bayofplenty,User-State_bc,User-State_be,User-State_bedfordshire,User-State_beijing,User-State_beiralitoral,User-State_belgium,User-State_belizedistrict,User-State_benalmdena,User-State_bergamo,User-State_bergued,User-State_berkshire,User-State_berlin,User-State_bern,User-State_berne,User-State_berneroberland,User-State_biella,User-State_bihor,User-State_bizkaia,User-State_bl,User-State_bohemia,User-State_bohusln,User-State_bolzano,User-State_bouchesdurhne,User-State_bourgogne,User-State_boxhillnorth,User-State_brabant,User-State_brabantwallon,User-State_braga,User-State_brandenburg,User-State_bremen,User-State_bretagne,User-State_brighton,User-State_bristol,User-State_britishcolumbia,User-State_brittany,User-State_brixton,User-State_brussels,User-State_bruxelles,User-State_bsas,User-State_buckinghamshire,User-State_buenosaires,User-State_bulacan,User-State_bulgaria,User-State_burgenland,User-State_burgos,User-State_bw,User-State_c,User-State_ca,User-State_caazapa,User-State_cadiz,User-State_cairo,User-State_cairogovernate,User-State_cairogovernorate,User-State_calabria,User-State_calfiornia,User-State_california,User-State_callif,User-State_cambridgeshire,User-State_campania,User-State_canada,User-State_canarias,User-State_canariasgrancanaria,User-State_cantabria,User-State_cantabury,User-State_cantaburynz,User-State_canterbury,User-State_capital,User-State_capitaldistrict,User-State_capitalfederal,User-State_carabobo,User-State_cascadia,User-State_cascais,User-State_cassablanca,User-State_castilla,User-State_castillalamancha,User-State_castillaylen,User-State_castillayleon,User-State_catalonia,User-State_catalua,User-State_catalunya,User-State_catalunyabarcelona,User-State_causewaybay,User-State_cavite,User-State_cceres,User-State_cdiz,User-State_cebu,User-State_central,User-State_centralamerica,User-State_centro,User-State_champagneardenne,User-State_champagne

In [6]:
# Step 3: Age column
# strip every non numeric characters
step2_data['User-Age'] = step2_data['User-Age'].astype(str).replace(r'[^\d]', "", regex=True)

# change to int, change to NaN otherwise
step2_data['User-Age'] = pd.to_numeric(step2_data['User-Age'], errors='coerce')

# now replace out of range ages
step2_data.loc[(step2_data['User-Age'] <= 0) | (step2_data['User-Age'] > 120), 'User-Age'] = np.nan

print("\nnumber of NaN after normalization\n",step2_data[['User-Age']].isna().sum())

# now, we drop the nan datas since it will ruin our model
step3_data = step2_data.dropna(subset="User-Age")

print("\nnumber of NaN after deletion\n",step3_data[['User-Age']].isna().sum())

step3_data.head()


number of NaN after normalization
 User-Age    18937
dtype: int64

number of NaN after deletion
 User-Age    0
dtype: int64


,User-ID,User-Age,User-State_aargau,User-State_aarhus,User-State_abelokipi,User-State_aberdeenshire,User-State_aberta,User-State_abilene,User-State_abruzzo,User-State_acorua,User-State_act,User-State_ae,User-State_ag,User-State_agbs,User-State_ahmedi,User-State_ain,User-State_ak,User-State_akershus,User-State_al,User-State_alabama,User-State_alacant,User-State_alajuela,User-State_alaska,User-State_alava,User-State_albacete,User-State_albay,User-State_alberta,User-State_alentejo,User-State_algarve,User-State_alicante,User-State_almada,User-State_almatyoblast,User-State_alpesmaritimes,User-State_alsace,User-State_altoalentejo,User-State_amsterdam,User-State_andaluca,User-State_andalucia,User-State_andalusia,User-State_andhrapradesh,User-State_angus,User-State_antarctica,User-State_antwerp,User-State_antwerpen,User-State_anywhereusa,User-State_aomoriken,User-State_aomoriprefecture,User-State_ap,User-State_apo,User-State_appenzell,User-State_aquitaine,User-State_ar,User-State_aragn,User-State_aragon,User-State_arizona,User-State_arkansas,User-State_armedforceseurope,User-State_armedforcespacific,User-State_armedserviceseurope,User-State_asia,User-State_asturias,User-State_asturies,User-State_athens,User-State_attika,User-State_attiki,User-State_auckland,User-State_australia,User-State_australiancapitalterritory,User-State_austria,User-State_auvergne,User-State_aveiro,User-State_avon,User-State_bacau,User-State_badajoz,User-State_badenwrrtemberg,User-State_badenwrtemberg,User-State_badenwrttemberg,User-State_badenwuerttemberg,User-State_bahia,User-State_bajacalifornia,User-State_bajacaliforniasur,User-State_baleares,User-State_bali,User-State_balkonien,User-State_bangkok,User-State_bankspeninsula,User-State_banstead,User-State_barcelona,User-State_barcelones,User-State_bari,User-State_barreiro,User-State_basel,User-State_baselland,User-State_baselstadt,User-State_basquecountry,User-State_basquecountryeuskalherria,User-State_basrhin,User-State_bassecote,User-State_bassenormandie,User-State_bavaria,User-State_bavariabayern,User-State_bayern,User-State_bayernbavaria,User-State_bayofplenty,User-State_bc,User-State_be,User-State_bedfordshire,User-State_beijing,User-State_beiralitoral,User-State_belgium,User-State_belizedistrict,User-State_benalmdena,User-State_bergamo,User-State_bergued,User-State_berkshire,User-State_berlin,User-State_bern,User-State_berne,User-State_berneroberland,User-State_biella,User-State_bihor,User-State_bizkaia,User-State_bl,User-State_bohemia,User-State_bohusln,User-State_bolzano,User-State_bouchesdurhne,User-State_bourgogne,User-State_boxhillnorth,User-State_brabant,User-State_brabantwallon,User-State_braga,User-State_brandenburg,User-State_bremen,User-State_bretagne,User-State_brighton,User-State_bristol,User-State_britishcolumbia,User-State_brittany,User-State_brixton,User-State_brussels,User-State_bruxelles,User-State_bsas,User-State_buckinghamshire,User-State_buenosaires,User-State_bulacan,User-State_bulgaria,User-State_burgenland,User-State_burgos,User-State_bw,User-State_c,User-State_ca,User-State_caazapa,User-State_cadiz,User-State_cairo,User-State_cairogovernate,User-State_cairogovernorate,User-State_calabria,User-State_calfiornia,User-State_california,User-State_callif,User-State_cambridgeshire,User-State_campania,User-State_canada,User-State_canarias,User-State_canariasgrancanaria,User-State_cantabria,User-State_cantabury,User-State_cantaburynz,User-State_canterbury,User-State_capital,User-State_capitaldistrict,User-State_capitalfederal,User-State_carabobo,User-State_cascadia,User-State_cascais,User-State_cassablanca,User-State_castilla,User-State_castillalamancha,User-State_castillaylen,User-State_castillayleon,User-State_catalonia,User-State_catalua,User-State_catalunya,User-State_catalunyabarcelona,User-State_causewaybay,User-State_cavite,User-State_cceres,User-State_cdiz,User-State_cebu,User-State_central,User-State_centralamerica,User-State_centro,User-State_champagneardenne,User-State_champagne